In [1]:
class read_sql_lab:
    def __init__(self,db_path):
        from sqlite3 import connect
        from pandas import read_sql
        
        conn = connect(db_path) #建立資料庫
        
        #透過SQL語法讀取資料庫中的資料
        # pd.read_sql(sql, con, index_col=None, coerce_float=True, params=None, parse_dates=None, columns=None, chunksize=None)
        self.job = read_sql("SELECT * FROM job", conn)
        self.user = read_sql("SELECT * FROM user", conn)[['id','username']]
        self.sample = read_sql("SELECT * FROM sample", conn)[['id','samplename','author_id','registered']]
        self.queue = read_sql("SELECT * FROM queue", conn)
        
    def list_samplename(self):
        self.tmp_samplelist = self.sample.sort_values(by = 'registered', ascending=False) 
        return list(self.tmp_samplelist['samplename'].unique())
    
    def update_list_samplename(self,keyword:str):
        if keyword == '':
            return self.list_samplename()
        self.list_samplename()
        self.tmp_samplelist = self.tmp_samplelist[self.tmp_samplelist['samplename'].str.contains(keyword, na=False)]
        return list(self.tmp_samplelist.sort_values(by = 'registered', ascending=False)['samplename'].unique())

    def select_sample(self,samplename):
#         print("Selected the sample : "+samplename)
        select_sample_id = self.tmp_samplelist[self.tmp_samplelist['samplename']==samplename]['id'].iloc[0]
        return select_sample_id
    
    def list_time(self,samplename=''):
        try:
            sample_id = sql.select_sample(samplename)
            return list(sql.job[sql.job['sample_id']==sample_id]['dateday'].unique())
        except:
            return list(sql.job['dateday'].unique())
    
#     def update_list_time(self,keyword:str):
#         tmp = self.job[self.job['dateday'].str.contains(keyword.split(r'(')[0], na=False)]
#         return list(tmp.sort_values(by = 'id', ascending=True)['dateday'].unique())

    def jobid_search_pyqum(self,id:int):
        sample_id = self.job[self.job['id']==id]['sample_id'].iloc[0]
        queue_name = self.job[self.job['id']==id]['queue'].iloc[0]
        dateday = self.job[self.job['id']==id]['dateday'].iloc[0]
        task = self.job[self.job['id']==id]['task'].iloc[0]
        wmoment  = self.job[self.job['id']==id]['wmoment'].iloc[0]
        name_id = self.sample[self.sample['id']==sample_id]['author_id'].iloc[0]
        name = self.user[self.user['id']==name_id]['username'].iloc[0]
        sample_name = self.sample[self.sample['id']==sample_id]['samplename'].iloc[0]
        mission = self.queue[self.queue['system']==queue_name]['mission'].iloc[0]
        pyqum_path = r"C:\Users\ASQUM\HODOR\CONFIG\USRLOG\%s\%s\%s\%s\%s.pyqum(%d)"%(name,sample_name,mission,dateday,task,int(wmoment))
        print("Pyqum Path : ",pyqum_path)
        return pyqum_path,task

# if __name__ == '__main__':
#     db_path = 'C:\\GitHub\\pyqum.sqlite'
#     sql = read_sql_lab(db_path)
#     sampleitem =sql.list_samplename()
#     print(sampleitem)

['NNCNB3-01-bTa-01', 'FDCNB3-Ta-Al-01', 'NCU-JPA-A1', '2QAS-19-3', '2QAS-19-4', '2QAS-17-13', 'ITRI_35_L1', 'ITRI_35_L2', 'ITRI_B_8955', 'ITRI_A_8955', '2QAS-17-10', '2QAS-12-08', 'NNCNB3-01-MBE-C5', '2QAS-11-07', '2QAS-12-06', '2QAS-11-08', '2QAS-12-05', 'WSNNL28-01-E9', 'NNCNB3-01-MBE-C7', 'NNCNB3-01-MBE-C8', 'NNCNB3-01-Ta-C8', 'WSNNL28-01-E1', '3SXQ-Al-Si-19-1_M2', 'WSNNL28-01-E8', '3SXQ-Al-Si-19-1', '3SXQ-Al-Si-19-4', 'NNCNB3-01-Ta-C4', 'NNCNB3-01-Ta-C5', 'WSNNL28-01-02', 'WSNNL28-01-01', 'DR-RFcable', 'NCU-Ctype-2QRB_v1', 'NNCNB3-01-Ta-02', 'NNCNB3-01-Ta-01', '3SXQ-Al-Si-17-2', '3SXQ-Al-Si-17-1', 'TWPA-NTHU', '2SXQ-MO-02', '2SXQ-MO-01', 'WSCNX16-02', 'WSCNX16-01', 'JPA-Al-Si-06-1', '3SXQ-Al-Si-17-4', '2D3Q-Ru', '3SXQ-16-1', '3SXQ-15-2', 'CPW-3Layer', 'CPW-Poly-1', 'CPW-5-8', '6-MDCNA01-01', '5-2Q-Al-Si-03-01', '4-XQ-Al-Si-11-02', '3-XQ-Al-Si-10-4', '2-XQ-Al-Si-10-3', '1-XQ-Al-Si-10-2', 'XQ-11-3', 'JPA-1', 'XQ-10-4', 'XQ-09-7', 'XQ-09-6', 'XQ-09-1', 'NadavA02', 'NadavA01', 'Chalmer

In [11]:
import pandas as pd
import tkinter as tk
from tkinter import ttk
import datetime

db_path = 'C:\\GitHub\\pyqum.sqlite'
sql = read_sql_lab(db_path)
window = tk.Tk()
window.title("PYQUM 地圖")
window.geometry("1280x720")

def samplename_update(*args):
    combo_category['values'] = []
    combo_category['values'] = sql.update_list_samplename(combo_category.get())
    combo_day['values'] = sql.list_time(combo_category.get())
    

label_category = tk.Label(window, text="Samplename：")
label_category.pack()
var1 = tk.StringVar()
combo_category = ttk.Combobox(window, values=sql.list_samplename(), textvariable=var1)
combo_category.pack()
var1.trace('w',samplename_update)


label_day = tk.Label(window, text="量測時間：")
label_day.pack()
var2 = tk.StringVar()
combo_day = ttk.Combobox(window, values=sql.list_time(), textvariable=var2)
combo_day.pack()

label_time = tk.Label(window, text="Hashtag：")
label_time.pack()
var3 = tk.StringVar()
times = []
combo_time = ttk.Combobox(window, values=times, textvariable=var3)
combo_time.pack()


def search(*args):
    tree.delete(*tree.get_children())  
    selected_category = var1.get()
    selected_day = var2.get()
    selected_time = var3.get()
    tmp = sql.job
    if var1.get() =='':
        pass
    else:
        sample_id = sql.select_sample(var1.get())
        tmp = tmp[tmp["sample_id"]==sample_id]
    if var2.get() =='':
        pass
    else:
        tmp = tmp[tmp['dateday']==var2.get()]
    
    
    if (var1.get() =='')&(var2.get() ==''):
        tree.insert("", "end", values=("","請選擇樣品或量測時間",  ""))
    else:
        if tmp.empty:
            tree.insert("", "end", values=("","沒有符合條件的情形", ""))
        else:
            for i in range(tmp.shape[0]):
                tree.insert("", "end", values=tuple(tmp.iloc[i, [0,4,10]].values))

button = tk.Button(text="搜尋", font=("Arial", 14, "bold"), padx=5, pady=5, bg="blue", fg="light green", command=search)
button.pack()

columns=("JOBID","Task","Comment")

tree=ttk.Treeview(window,show="headings",columns=columns)
tree.column("JOBID",width=100,anchor='center')
tree.column("Task",width=200,anchor='center')
tree.column("Comment",width=600,anchor='center')
tree.heading("JOBID",text="JOBID")
tree.heading("Task",text="Task")
tree.heading("Comment",text="Comment")
tree.pack()

window.mainloop()